### 人类活动识别

In [ ]:
from model.RFID_Dataset import RFID_Dataset
from utils.DatasetUtils import DatasetUtils
from torch.utils.data import ConcatDataset


data_utils = DatasetUtils()

from torchvision import transforms
from model.Normalization import *

transform=transforms.Compose([
    RobustNorm(-35.0, 35.0)
])

train_dir = r"data\RFID\dataset\train"
eval_dir = r"data\RFID\dataset\eval"
generate_dir=r"./output/merge"

# 加载数据集
train_dataset = RFID_Dataset(
    train_dir,
    T=32,
    step=1,
    transform=transform,
)

eval_dataset = RFID_Dataset(
    eval_dir,
    T=32,
    step=1,
    transform=transform,
)

# generate_dataset=RFID_Dataset(
#     generate_dir,
#     T=32,
#     step=32,
#     # transform=transform,
# )

# # 拼接数据集
# integrated_dataset=ConcatDataset([
#     train_dataset,
#     generate_dataset
# ])

# train_dataset=integrated_dataset

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 模型组网
from torchkeras import summary
from model.ClassifyNet.CNNClassifyNet import CNNClassifyNet as ClassifyNet
from model.ModelWorker.ClassifyModelWorker import ClassifyModelWorker

input_shape=(1,32,12)
model=ClassifyNet(
    input_shape=input_shape,
    num_classes=3
)

model_worker=ClassifyModelWorker(model)

print(f"{input_shape=}")
model_info=summary(model,input_shape=input_shape)

In [ ]:
# 模型准备
from torch.utils.data import DataLoader
from torch import nn
from torch import optim

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=4)

loss = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005)


In [ ]:
# 模型训练

model_worker.train(
    criterion=loss,
    optimizer=optimizer,
    train_loader=train_loader,
    eval_loader=eval_loader,
    epochs=5,
    enable_board=False,
)

In [ ]:
# 模型评估

model_worker.evaluate(
    eval_loader=eval_loader,
    criterion=loss,
)

In [ ]:
# 评价指标

from torchmetrics import MetricCollection, Accuracy, F1Score,ConfusionMatrix

res=model_worker.execute_metric(
    eval_loader, 
    MetricCollection([
        # Accuracy(task="multiclass", num_classes=10),
        F1Score(task="multiclass", num_classes=3),
        # ConfusionMatrix(task="multiclass", num_classes=10)
    ]),
)

print(res)

In [ ]:
# 保存模型
model_worker.save('./output/HAR/HAR_robust.pth')

In [ ]:
# 加载模型
model_worker.load('./output/HAR/HAR.pth')

In [ ]:
from torch.utils.data import DataLoader


test_dataset=RFID_Dataset(
    r"./output/merge",
    T=32,
    step=32,
    # transform=transform,
)
print(f"测试集的数据个数: {len(test_dataset)}")

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=8)
model_worker.evaluate(
    eval_loader=test_loader,
    criterion=loss,
)

In [ ]:
# 模型预测

used_dataset = eval_dataset
indexs,inputs,labels = data_utils.select_simple(used_dataset,count=4)
preds,acc = model_worker.predict(inputs=inputs,labels=labels)

print(f"{inputs.shape=}")
print(f"{acc=}")
print(f"indexs:{indexs}")
print(f"labels:{labels}")
print(f"preds :{preds}")